In [1]:
# Copyright 2020 NVIDIA Corporation. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

<img src="http://developer.download.nvidia.com/compute/machine-learning/frameworks/nvidia_logo.png" style="width: 90px; float: right;">

## Overview
The last step is to deploy the ETL workflow and saved model to production. In the production setting, we want to transform the input data as done during training ETL. We need to apply the same mean/std for continuous features and use the same categorical mapping to convert the categories to continuous integers before we use the deep learning model for a prediction. Therefore, we deploy the NVTabular workflow with the TensorFlow model as an ensemble model to Triton Inference. The ensemble model guarantees that the same transformation are applied to the raw inputs.

<center><img src="./imgs/triton-tf.png" width="300" height="200"></center>

## Learning Objectives
In the previous notebook we explained and showed how we can preprocess data with multi-hot columns with NVTabular, and train an TF MLP model using NVTabular KerasSequenceLoader. We learned how to save a workflow, a trained TF model, and the ensemble model. In this notebook, we will show an example request script sent to triton inference server

- to transform new/streaming data with NVTabular library
- to deploy the end-to-end pipeline to generate prediction results for new data from trained TF model

## Getting Started

In [2]:
# External dependencies
import os
from os import path                
from time import time
import gc

import tensorflow as tf
import nvtabular as nvt
import cudf 
from tritonclient.utils import *
import tritonclient.grpc as grpcclient
import nvtabular.inference.triton as nvt_triton



We define our base directory, containing the data.

In [3]:
# path to store raw and preprocessed data
INPUT_DATA_DIR = os.environ.get('INPUT_DATA_DIR', os.path.expanduser("~/nvt-examples/movielens/data/"))
# path to save models
MODEL_BASE_DIR = os.environ.get('MODEL_BASE_DIR', '/model/')

Let's deactivate the warnings before sending requests.

In [4]:
import warnings
warnings.filterwarnings('ignore')

First, we need to generate the Triton Inference Server configurations and save the models in the correct format. In the previous notebooks 02-ETL-with-NVTabular and 03a-Training-with-TF we saved the NVTabular workflow and TensorFlow model to disk. We will load them first.

In [5]:
workflow = nvt.Workflow.load(os.path.join(INPUT_DATA_DIR, "workflow"))


workflow.output_dtypes["userId"] = "int32"
workflow.output_dtypes["movieId"] = "int32"
MODEL_NAME_TF = os.environ.get("MODEL_NAME_TF", "movielens_tf")

In [6]:
model = tf.keras.models.load_model(os.path.join(MODEL_BASE_DIR, MODEL_NAME_TF, "1/model.savedmodel"))

In [7]:
MODEL_NAME_ENSEMBLE = os.environ.get("MODEL_NAME_ENSEMBLE", "movielens")
# model path to save the models
MODEL_PATH = os.environ.get('MODEL_PATH', os.path.join(MODEL_BASE_DIR, "models"))

NVTabular provides an easy function to deploy the ensemble model for Triton Inference Server.

In [8]:
# Creates an ensemble triton server model, where
#   model: The tensorflow model that should be served
#   workflow: The nvtabular workflow used in preprocessing
#   name: The base name of the various triton models

from nvtabular.inference.triton import export_tensorflow_ensemble
export_tensorflow_ensemble(model, workflow, MODEL_NAME_ENSEMBLE, MODEL_PATH, ["rating"])

INFO:tensorflow:Assets written to: /model/models/movielens_tf/1/model.savedmodel/assets


## Starting Triton Server

At this staged, you should have already launched the Triton Inference Server docker container with the following script. This mounts your local `model-repository` folder where you stored the models into the `merlin-inference` docker container.

```
docker run -it --gpus device=0 -p 8000:8000 -p 8001:8001 -p 8002:8002 -v ${PWD}:/model/ nvcr.io/nvidia/merlin/merlin-inference:0.5

source activate merlin
```

After you started the container you can start triton server with the command below. You need to provide correct path for the `models` directory.

```
tritonserver --model-repository=path_to_models --backend-config=tensorflow,version=2 --model-control-mode=explicit 
```

Note: The model-repository path is `/model/models/`. The models haven't been loaded, yet. Below, we will request the Triton server to load the saved ensemble model.

## Loading Ensemble Model with Triton Inference Serve

At this stage, you should have launched the Triton Inference Server docker container with the instructions above.

Let's connect to the Triton Inference Server. Use Triton’s ready endpoint to verify that the server and the models are ready for inference. Replace localhost with your host ip address.

In [9]:
import tritonhttpclient
try:
    triton_client = tritonhttpclient.InferenceServerClient(url="localhost:8000", verbose=True)
    print("client created.")
except Exception as e:
    print("channel creation failed: " + str(e))

client created.


/opt/conda/envs/merlin/lib/python3.8/site-packages/tritonhttpclient/__init__.py:30: DeprecationWarning: The package `tritonhttpclient` is deprecated and will be removed in a future version. Please use instead `tritonclient.http`
  warnings.warn(


In [10]:
import warnings
warnings.filterwarnings('ignore')

/opt/conda/envs/merlin/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


We check if the server is alive.

In [11]:
triton_client.is_server_live()

GET /v2/health/live, headers None
<HTTPSocketPoolResponse status=200 headers={'content-length': '0', 'content-type': 'text/plain'}>


True

The HTTP request returns status 200 if Triton is ready and non-200 if it is not ready.

We check the available models in the repositories:

movielens: Ensemble <br>
movielens_nvt: NVTabular <br>
movielens_tf: TensorFlow model

In [12]:
triton_client.get_model_repository_index()

POST /v2/repository/index, headers None

<HTTPSocketPoolResponse status=200 headers={'content-type': 'application/json', 'content-length': '71'}>
bytearray(b'[{"name":"movielens"},{"name":"movielens_nvt"},{"name":"movielens_tf"}]')


[{'name': 'movielens'}, {'name': 'movielens_nvt'}, {'name': 'movielens_tf'}]

We load the ensemble model.

In [13]:
%%time

triton_client.load_model(model_name='movielens')

POST /v2/repository/models/movielens/load, headers None

<HTTPSocketPoolResponse status=200 headers={'content-type': 'application/json', 'content-length': '0'}>
Loaded model 'movielens'
CPU times: user 11.7 ms, sys: 5.35 ms, total: 17 ms
Wall time: 10.6 s


## Send request to Triton IS to transform raw dataset

A minimal model repository for a TensorFlow SavedModel model is:
```
  <model-repository-path>/<model-name>/
      config.pbtxt
      1/
        model.savedmodel/
           <saved-model files>
```
Let's check out our model repository layout. You can install tree library with apt-get install tree, and then run `!tree /model/models/` to print out the model repository layout as below:
               
```
/model/models/
|-- movielens
|   |-- 1
|   `-- config.pbtxt
|-- movielens_nvt
|   |-- 1
|   |   |-- __pycache__
|   |   |   `-- model.cpython-38.pyc
|   |   |-- model.py
|   |   `-- workflow
|   |       |-- categories
|   |       |   |-- unique.genres.parquet
|   |       |   |-- unique.movieId.parquet
|   |       |   `-- unique.userId.parquet
|   |       |-- metadata.json
|   |       `-- workflow.pkl
|   `-- config.pbtxt
`-- movielens_tf
    |-- 1
    |   `-- model.savedmodel
    |       |-- assets
    |       |-- saved_model.pb
    |       `-- variables
    |           |-- variables.data-00000-of-00001
    |           `-- variables.index
    `-- config.pbtxt
```
You can see that we have a `config.pbtxt` file. Each model in a model repository must include a model configuration that provides required and optional information about the model. Typically, this configuration is provided in a `config.pbtxt` file specified as [ModelConfig protobuf](https://github.com/triton-inference-server/server/blob/r20.12/src/core/model_config.proto).

Let's read the raw validation set, and send 3 rows of `userId` and `movieId` as input to the saved NVTabular model.

In [14]:
# read in the workflow (to get input/output schema to call triton with)
batch = cudf.read_parquet(os.path.join(INPUT_DATA_DIR, "valid.parquet"), num_rows=3, columns=['userId', 'movieId'])
print(batch)

          userId  movieId
15347762   99476   104374
16647840  107979     2634
23915192  155372     1614


In [15]:
inputs = nvt_triton.convert_df_to_triton_input(["userId", "movieId"],batch, grpcclient.InferInput)

outputs = [
    grpcclient.InferRequestedOutput(col)
    for col in ["userId", "movieId", "genres__nnzs", "genres__values"]
]

MODEL_NAME_NVT = os.environ.get('MODEL_NAME_NVT', 'movielens_nvt')

with grpcclient.InferenceServerClient("localhost:8001") as client:
    response = client.infer(MODEL_NAME_NVT, inputs, request_id="1", outputs=outputs)

for col in ["userId", "movieId", "genres__nnzs", "genres__values"]:
    print(col, response.as_numpy(col), response.as_numpy(col).shape)

userId [[ 99476]
 [107979]
 [155372]] (3, 1)
movieId [[19997]
 [ 2543]
 [ 1557]] (3, 1)
genres__nnzs [[3]
 [1]
 [1]] (3, 1)
genres__values [[ 9]
 [10]
 [16]
 [12]
 [ 6]] (5, 1)


You might notice that we don't need to send the genres column as an input. The reason for that is the nvt model will look up the genres for each movie as part of the `JoinExternal` op it applies. Also notice that when creating the request for the `movielens_nvt` model, we return 2 columns (values and nnzs) for the `genres` column rather than 1.

## END-2-END INFERENCE PIPELINE

We will do the same, but this time we directly read in first 3 rows of the the raw `valid.parquet` file with cuDF.

In [16]:
# read in the workflow (to get input/output schema to call triton with)
batch = cudf.read_parquet(os.path.join(INPUT_DATA_DIR, "valid.parquet"), num_rows=3, columns=['userId', 'movieId'])

print("raw data:\n", batch, "\n")

# convert the batch to a triton inputs
inputs = nvt_triton.convert_df_to_triton_input(["userId", "movieId"], batch, grpcclient.InferInput)

# placeholder variables for the output
outputs = [grpcclient.InferRequestedOutput("output")]

MODEL_NAME_ENSEMBLE = os.environ.get('MODEL_NAME_ENSEMBLE', 'movielens')

# build a client to connect to our server. 
# This InferenceServerClient object is what we'll be using to talk to Triton.
# make the request with tritonclient.grpc.InferInput object

with grpcclient.InferenceServerClient("localhost:8001") as client:
    response = client.infer(MODEL_NAME_ENSEMBLE, inputs, request_id="1",outputs=outputs)

print("predicted sigmoid result:\n", response.as_numpy('output'))


raw data:
           userId  movieId
15347762   99476   104374
16647840  107979     2634
23915192  155372     1614 

predicted sigmoid result:
 [[0.6316925]
 [0.6214284]
 [0.6092132]]


Let's send request for a larger batch size and measure the total run time and throughput.

In [17]:
# read in the workflow (to get input/output schema to call triton with)
batch_size = 64
batch = cudf.read_parquet(os.path.join(INPUT_DATA_DIR, "valid.parquet"), num_rows=batch_size, columns=['userId', 'movieId'])

start = time()
# convert the batch to a triton inputs
inputs = nvt_triton.convert_df_to_triton_input(["userId", "movieId"], batch, grpcclient.InferInput)

# placeholder variables for the output
outputs = [grpcclient.InferRequestedOutput("output")]

MODEL_NAME_ENSEMBLE = os.environ.get('MODEL_NAME_ENSEMBLE', 'movielens')

# build a client to connect to our server. 
# This InferenceServerClient object is what we'll be using to talk to Triton.
# make the request with tritonclient.grpc.InferInput object

with grpcclient.InferenceServerClient("localhost:8001") as client:
    response = client.infer(MODEL_NAME_ENSEMBLE, inputs, request_id="1",outputs=outputs)

t_final = time() - start
print("predicted sigmoid result:\n", response.as_numpy('output'), "\n")

print(f"run_time(sec): {t_final} - rows: {batch_size} - inference_thru: {batch_size / t_final}")

predicted sigmoid result:
 [[0.6316925 ]
 [0.6214284 ]
 [0.6092132 ]
 [0.6235167 ]
 [0.6250084 ]
 [0.6395785 ]
 [0.62557054]
 [0.61516464]
 [0.6292164 ]
 [0.6211892 ]
 [0.6149823 ]
 [0.6212783 ]
 [0.63796204]
 [0.62901783]
 [0.6384243 ]
 [0.62431633]
 [0.6191272 ]
 [0.6165652 ]
 [0.630915  ]
 [0.6159394 ]
 [0.6147852 ]
 [0.62025106]
 [0.6266043 ]
 [0.6254384 ]
 [0.6317272 ]
 [0.6163654 ]
 [0.6379906 ]
 [0.61515933]
 [0.6258164 ]
 [0.6195812 ]
 [0.6080455 ]
 [0.62915725]
 [0.6109987 ]
 [0.6166749 ]
 [0.6099736 ]
 [0.6136805 ]
 [0.61030835]
 [0.6095582 ]
 [0.6122199 ]
 [0.6205388 ]
 [0.6159254 ]
 [0.63010496]
 [0.6215024 ]
 [0.6130587 ]
 [0.61035496]
 [0.61039805]
 [0.6098601 ]
 [0.61884046]
 [0.62948465]
 [0.6334557 ]
 [0.6185863 ]
 [0.61159116]
 [0.6284263 ]
 [0.6196071 ]
 [0.61471736]
 [0.6314106 ]
 [0.624711  ]
 [0.61086106]
 [0.61007977]
 [0.6313912 ]
 [0.62896764]
 [0.6167822 ]
 [0.610881  ]
 [0.61704606]] 

run_time(sec): 0.06319451332092285 - rows: 64 - inference_thru: 1012.74614

Let's unload all the models.

In [19]:
triton_client.unload_model(model_name='movielens')
triton_client.unload_model(model_name='movielens_nvt')
triton_client.unload_model(model_name='movielens_tf')

POST /v2/repository/models/movielens/unload, headers None

<HTTPSocketPoolResponse status=200 headers={'content-type': 'application/json', 'content-length': '0'}>
Loaded model 'movielens'
POST /v2/repository/models/movielens_nvt/unload, headers None

<HTTPSocketPoolResponse status=200 headers={'content-type': 'application/json', 'content-length': '0'}>
Loaded model 'movielens_nvt'
POST /v2/repository/models/movielens_tf/unload, headers None

<HTTPSocketPoolResponse status=200 headers={'content-type': 'application/json', 'content-length': '0'}>
Loaded model 'movielens_tf'
